In [5]:
import os
os.chdir('../src')
os.getcwd()

'/Users/kolya/Documents/Skoltech_PhD/courses/Theoretical Foundations of Data Science/project/OSM-one-shot-multispeaker/src'

In [6]:
from tts_modules.common.multispeaker import MultispeakerManager
import yaml

In [10]:
with open(os.path.join(os.getcwd(), 'tts_modules/encoder/configs/AudioConfig.yaml'), "r") as ymlfile:
    AudioConfig = yaml.load(ymlfile)
AudioConfig['MEL_N_CHANNELS']

<ipython-input-10-139d9a300b26>:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  AudioConfig = yaml.load(ymlfile)


25